The objective of this demo is to showcase a scenario in which an hypothetical company wants to implement an AI system to predict which employees should be targeted for retention actions, based on various features of the employee. The AI system should be trustable, hence they want to use trustAI to assess the trustworthiness of the candidate classification models before their deployment and use for decision making.

We use the synthetic dataset TED to simulate the scenario and the trustAI package to verify the model's trustworthiness. Unlike most datasets, each instance has also have an Explanation (E). Therefore, we will use the TED Cartesian Explainer as the explainer's framework, and the associated trustworthiness explainability metric included in the trustAI package.

# Step 1: Defining the configuration file
In this scenario, we are interested in the explainability trustworthiness dimension, as the company requires an AI system that provide explanations of why a retention action is needed to reduce the chances of an employee leaving the company. These rules are motivated by common scenarios, such as not getting a promotion in a while, not being paid competitively, receiving a disappointing evaluation, being a new employee in certain organizations with inherently high attrition, not having a salary that is consistent with positive evaluations, mid-career crisis, etc.

The company is also interested in the performance trustworthiness dimension, as predicting accurately which employees should be targeted for retention (human resources) is equally important to predicting which ones should not be targeted for retention (budget restrictions).

Based on this, we specify a configuration file based on metrics belonging to the two trustworthiness dimensions, and we specify the assessment method as a weighted average with equal weights for the two dimensions and the metrics that will be used.

```yaml
metrics:
    - AccuracySKL
    - PrecisionSKL:
        multiclass_average: "binary"
    - RecallSKL:
        multiclass_average: "binary"
    - F1SKL:
        multiclass_average: "binary"
    - ROCSKL:
        multiclass_average: "macro"
    - ExplanationsAccuracyTED:
        explainer_path: "demos/employees_retention/ted_explainer"
        explanations_path: "demos/employees_retention/ted_explanations"
assessment_method:
    WeightedAverage: # WEIGHTS FOR COMPUTING TRUST WITH THE WEIGHTED AVERAGED METHOD
        performance-0.5:
            AccuracySKL: 0.2
            PrecisionSKL: 0.2
            RecallSKL: 0.2
            F1SKL: 0.2
            ROCSKL: 0.2                
        explainability-0.5:                
            ExplanationsAccuracyTED: 1
```

# Step 2: Import relevant packages

The first step will consist in importing the trustAI package, the classification model that we will use in the demo (RandomForestClassifier), the TED Cartesian Explainer that will be trained and used to explain the model's predictions and some supporting functions/modules, notably pandas for the dataset loading/manipulation, and train_test_split to partition the dataset.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from aix360.algorithms.ted.TED_Cartesian import TED_CartesianExplainer
from TrustComputationService.TrustComputationService import TrustComputationService
import pandas as pd
from pickle import dump

We force pandas to print every column of the dataset when needed.

In [ ]:
pd.reset_option("max_columns")
pd.set_option('display.max_columns', None)

# Step 2: Load dataset and create train/test splits
Below we define variables containing the file paths to the relevant files needed for this use case: the CSV dataset file and the previously defined configuration file.

We then load the dataset using pandas, and we separate the features, the target, and the explanations (NOTE: Code partially extracted from https://github.com/Trusted-AI/AIX360/blob/master/aix360/datasets/ted_dataset.py). 

We also replace the values of the target column from -10, -11 to the more interpretable 0, 1 values. 

Finally, we create train/test splits for the features, target, and explainations, with 80%-20% train-test, correspondingly,

In [ ]:
demo_path = 'demos/employees_retention/'
dataset_path = demo_path + 'Retention.csv'
configuration_path = demo_path + 'config_retention_corporate.yml'

# Decompose the dataset into X, Y, E
data = pd.read_csv(dataset_path)
X = data.iloc[:,:-2]   # Choose all rows and all cols, except for the last 2 cols
Y = data['Y']          # Choose col with header 'Y'
E = data['E']          # Choose col with header 'E'

# CLASSIFICATION COLUMN. [-10,-11] -> [0,1]
Y.replace([-10,-11], [0,1], inplace=True)

# Set up train/test split
X_train, X_test, Y_train, Y_test, E_train, E_test = train_test_split(X, Y, E, test_size=0.3, random_state=1)

# Step 3: Create the classifier and the TED classifier
Now we will create a random forest classifier and a TED-enhanced cartesian classifer, which will take the random forest classifier and will be trained taking into account the traininig explanations, i.e., the TED-enhanced classifier takes the training features and, for the target, it merges the actual targets (Y) and the explanations (E). Therefore, we will train a "regular" random forest classifier too, in order to evaluate the performance-related trustworthiness metrics on the Y targets.

In [ ]:
# TRAIN A RANDOM FOREST CLASSIFIER AND CREATE A TED-ENHANCED CLASSIFIER
rf_classifier = RandomForestClassifier()
ted = TED_CartesianExplainer(rf_classifier)

rf_classifier_raw = RandomForestClassifier().fit(X_train, Y_train)
ted.fit(X_train, Y_train, E_train)

# Step 4: Store required assets
The explainability-related metric *ExplanationsAccuracyTED* requires two additional properties, a TED-enhanced classifier, and a dataset of explanations. Therefore, we will store such assets into the directory we have set in the configuration file. The "ted_explainer" property corresponds to the previously fitted TED-enhanced classifier, and the "ted_explanations" corresponds to the explanations we will test the classifier with, i.e., the test explanations. We store both using pickle.

In [ ]:
# SAVE EXPLAINER AND EXPLANATIONS TO THE DEMO DIRECTORY (OPTIONAL, ONLY NECESSARY IF THEY ARE NOT ALREADY PRESENT)
with open(demo_path + 'ted_explainer', 'wb') as explainer_file:
    dump(ted, explainer_file)
with open(demo_path + 'ted_explanations', 'wb') as explanations_file:
    dump(E_test, explanations_file)

# Step 5: Compute the trustworthiness
Now we are ready to assess the model's trustworthiness with the *trustAI* package. For this, we instantiate a `TrustComputationService`, we call the `readTrustDefinition` method with the path to the configuration file we specified, and lastly we call the `computeTrust` function, passing the trained model and the test dataset (features and target) to evaluate the model's trustworthiness in such dataset. This function will return the trust assessment (in this case a float, as we use the weighted average assessment method).

In [ ]:
trust_instance = TrustComputationService()
trust_instance.readTrustDefinition(configuration_path)
print(trust_instance.computeTrust(rf_classifier_raw, X_test, Y_test))

Which results in a value of 0.99 of the trustworthiness indicator for the classification model that will conform the AI system.

We can also print the complete trustworthiness assessment as a string:

In [ ]:
print(trust_instance.getTrustAssessmentAsFormattedString())

```javascript
{
  "name": "Trust",
  "weighted_score": 0.99,
  "children": [
    {
      "name": "performance",
      "weight": 0.5,
      "weighted_score": 0.5,
      "raw_score": 0.99,
      "children": [
        {
          "name": "AccuracySKL",
          "weight": 0.2,
          "weighted_score": 0.2,
          "raw_score": 0.99
        },
        {
          "name": "PrecisionSKL",
          "weight": 0.2,
          "weighted_score": 0.2,
          "raw_score": 0.99
        },
        {
          "name": "RecallSKL",
          "weight": 0.2,
          "weighted_score": 0.2,
          "raw_score": 1.0
        },
        {
          "name": "F1SKL",
          "weight": 0.2,
          "weighted_score": 0.2,
          "raw_score": 0.99
        },
        {
          "name": "ROCSKL",
          "weight": 0.2,
          "weighted_score": 0.2,
          "raw_score": 0.99
        }
      ]
    },
    {
      "name": "explainability",
      "weight": 0.5,
      "weighted_score": 0.49,
      "raw_score": 0.98,
      "children": [
        {
          "name": "ExplanationsAccuracyTED",
          "weight": 1,
          "weighted_score": 0.98,
          "raw_score": 0.98
        }
      ]
    }
  ]
}
```

# Conclusions
This notebook has illustrated how easy it is to use the trustAI package to evaluate the trustworthiness of a classification model intended to be used as part of an AI system. In this case, the trustAI package has been used as part of a model building pipeline, obtaining a positive trustworthiness assessment (i.e., 0.99 out of 1). According to the trustability criteria of the ficticious company, the model would be deemed as acceptable without the need of undergoing modifications.